In [334]:
# import libraries
import cv2
from keras.datasets import cifar10

import numpy as np
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import keras
from keras import layers
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from keras.models import Model
from keras.layers import Input, Dense


In [335]:
# load the data
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [336]:
print(trainX.shape)
print(testX.shape)
# trainY is (50000,1) and they are labels
print(trainY.shape)
print(testY.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [337]:
# Color to Gray
x_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in trainX])
x_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in testX])

In [338]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32)
(10000, 32, 32)


In [339]:
# normalize the images
x_train = x_train/255.0
x_test = x_test/255.0

# get the shape of data after normalization
print(x_train.shape)

(50000, 32, 32)


In [340]:
# reshape 50000*32*32 to 50000*1042
x_train = np.reshape(x_train,(50000,1024))
x_test = np.reshape(x_test,(10000,1024))
# print the shape of x_train and x_test after reshape
print(x_train.shape)
print(x_test.shape)

(50000, 1024)
(10000, 1024)


In [341]:
def initial_centroids(length, K):
  idx = np.random.choice(length,K,replace = False)
  return idx

In [342]:
def mean_function(centroid_index, initil_centroids_index, data):
  mean_value = data[centroid_index==initil_centroids_index].mean(axis=0)
  return mean_value

In [343]:
def minimum_distance_centroid(centroid, data):
  distance = cdist(data, centroid ,'euclidean')
  minimum_distance = np.array([np.argmin(i) for i in distance])
  return minimum_distance

In [344]:
def kmeans_function(data, iterations):
    index = initial_centroids(len(data), 10)
    #Choosing Random Centroids 
    centroids = data[index, :]
    min_dist_centroid = minimum_distance_centroid(centroids,data)
    for _ in range(iterations): 
        centroids = []
        for index in range(10):
          index = 0
          while index < 10:
            temp_centroid = mean_function(min_dist_centroid,index,data) 
            centroids.append(temp_centroid)
            index += 1
        centroids = np.vstack(centroids) 
        min_dist_centroid = minimum_distance_centroid(centroids,data)   
    return min_dist_centroid 

In [345]:
kmean_scratch = kmeans_function(x_test,100)
silhouette_score1 = silhouette_score(x_test, kmean_scratch)
print(silhouette_score1)

0.053588923228439715


In [346]:
# Part 2

In [347]:
# Auto encoders

input_layer = Input(shape=(1024,))
encoded_layer = Dense(512, activation='relu')(input_layer)
bottleneck = Dense(256, activation = 'relu')(encoded_layer)
decoded_layer = Dense(512, activation='linear')(bottleneck)
output_layer = Dense(1024, activation='linear')(decoded_layer)

autoencoder_model = Model(input_layer, output_layer)

In [348]:
autoencoder_model.compile(optimizer='adam', loss='mean_squared_error')

In [349]:
autoencoder_model.fit(x_train, x_train,
                epochs=10,
                batch_size=1024)

Epoch 1/10
49/49 [==============================] - 1s 13ms/step - loss: 0.0563
Epoch 2/10
49/49 [==============================] - 1s 12ms/step - loss: 0.0252
Epoch 3/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0193
Epoch 4/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0163
Epoch 5/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0153
Epoch 6/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0135
Epoch 7/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0125
Epoch 8/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0117
Epoch 9/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0112
Epoch 10/10
49/49 [==============================] - 1s 11ms/step - loss: 0.0108


In [350]:
predict1 = autoencoder_model.predict(x_train)

In [351]:
encoder_model_for_kmeans = Model(input_layer, bottleneck)

In [352]:
predict2 = encoder_model_for_kmeans.predict(x_train)

In [353]:
scikit_KMeans = KMeans(n_clusters = 10,random_state = 0).fit(predict2)


In [354]:
silhouette_score = silhouette_score(predict2, scikit_KMeans.labels_)
print(silhouette_score)

0.07771354
